In [15]:
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import copy
import torch as tr

# import scripts
from src.Botorch_opt import bayesian_optimiser
from src.api_helper import api_utils
from src.api_helper import env
from datetime import datetime as dt, date as dt_date

dtype = tr.float32

##### data

In [16]:
@api_utils.transform
def api(query): return env.rosenbrock(query)

x0, y0 = tr.rand(2).view(1, -1), tr.tensor([[-1]], dtype=tr.float).view(-1, 1)  
r0 = env.rosenbrock(x0)  # initial reward on which normalisation based

print("init query", x0); print("init reward", api(x0, r0))

init query tensor([[0.3059, 0.8652]])
init reward tensor([[-1.]])


##### OPT

In [17]:
# hyperparameters
T = 5  # total number of iterations

# gp; includes "MA2.5", "SE", "RQ", "LR", "PO"
gp_name, gp_params = "RQ",{
                          "mode": "pro",      # "raw", "add", "pro" for GPs
                          "opt":"ADAM",        # opt for MLE; (quasi_newton, ADAM)
                          "epochs":128,       # epoch to run, if chosen ADAM
                          "lr":1e-1,          # learning rate for ADAM
                         }
# q-parallelism (if use analytical acq_func, q must be 1)
batch_size = 1

acq_params = { 
    "acq_name" : "UCB",          # acqu func; includes: "EI", "UCB", "qEI", "qUCB", "qKG"
    "N_start": 32,               # number of starts for multi-start SGA
    "raw_samples" :512,          # heuristic initialisation 
    "N_MC_sample" : 256,         # number of samples for Monte Carlo simulation
    "num_fantasies": 128,        # number of fantasies used by KG
    "beta":1.,                   # used by UCB/qUCB
               }

bayes_opt = bayesian_optimiser(gp_name, gp_params, acq_params)

In [18]:
%%time
xs, ys = bayes_opt.outer_loop(T, (0,3),x0, y0, r0, api, batch_size)

Iter 128 - Loss: -3.641
iteration: 1, reward: -0.93
Iter 128 - Loss: -1.871
iteration: 2, reward: -101.17
Iter 128 - Loss: 9232.153
iteration: 3, reward: -0.91
Iter 128 - Loss: 173.768
iteration: 4, reward: -0.30
Iter 128 - Loss: 15.194
iteration: 5, reward: -0.06
CPU times: user 4.77 s, sys: 9.31 s, total: 14.1 s
Wall time: 3.55 s
